# Benchmark m4

How to fit several model and evaluate its predictions on three of subsets of M4 data. 


#### functools.partial

Return a new `partial` object which when called will behave like *func* called with the positional arguments args and keyword arguments keywords. 

- [Source](https://docs.python.org/2/library/functools.html)

## Methods  

Brief overview of the used methods

### `DeepAREstimator`

- [`gluonts.model.deepar`](https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html)

Construct a DeepAR estimator, implements an RNN-based model, close to the one described in

- Salinas, David, Valentin Flunkert, and Jan Gasthaus. “DeepAR: Probabilistic forecasting with autoregressive recurrent networks.” arXiv preprint arXiv:1704.04110 (2017).


### `MQCNNEstimator`

- [`gluonts.model.seq2seq`](http://gluon-ts.mxnet.io/api/gluonts/gluonts.model.seq2seq.html)

An `MQDNNEstimator` with Convolutional Neural Network (CNN) as an encoder. Implements the MQ-CNN Forecaster, proposed in 

- Wen, Ruofeng, et al. “A multi-horizon quantile recurrent forecaster.” arXiv preprint arXiv:1711.11053 (2017).


### `SimpleFeedForwardEstimator`

- [`gluonts.model.simple_feedforward`](http://gluon-ts.mxnet.io/api/gluonts/gluonts.model.simple_feedforward.html)

A simple multilayer perceptron model predicting the next target time-steps given the previous ones. 



In [1]:
# install gluonts (most recent version)
# !pip install git+https://github.com/awslabs/gluon-ts.git

In [2]:
# imports
import numpy as np
import mxnet as mx

import pandas as pd

import pprint
from functools import partial

# gluonts imports
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.distribution.piecewise_linear import PiecewiseLinearOutput
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.trainer import Trainer

# gluonts models
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.seq2seq import MQCNNEstimator
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator


INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU


In [3]:
# gluonts version (important)
import gluonts
print(gluonts.__version__)

0.4.0


Number of series in m4 data: 

- Yearly: 23,000
- Quarterly: 24,000
- Monthly: 48,000
- Weekly: 359
- Daily: 4227
- Hourly: 414



In [28]:
epochs = 200
num_batches_per_epoch = 50

datasets = [
#     "m4_hourly",
#     "m4_daily",
#     "m4_weekly",
#     "m4_monthly",
    "m4_quarterly",
    "m4_yearly",
]
print(datasets)

['m4_quarterly', 'm4_yearly']


In [29]:
estimators = [
#     partial(
#         SimpleFeedForwardEstimator,
#         trainer=Trainer(
#             epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
#         ),
#     ),
    partial(
        DeepAREstimator,
        trainer=Trainer(
            epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
        ),
    ),
#     partial(
#         DeepAREstimator,
#         distr_output=PiecewiseLinearOutput(8),
#         trainer=Trainer(
#             epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
#         ),
#     ),
#     partial(
#         MQCNNEstimator,
#         trainer=Trainer(
#             epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
#         ),
#     ),
]


INFO:root:Using CPU


In [30]:
def evaluate(dataset_name, estimator):
    dataset = get_dataset(dataset_name)
    estimator = estimator(
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True,
        cardinality=[
            feat_static_cat.cardinality
            for feat_static_cat in dataset.metadata.feat_static_cat
        ],
    )

    print(f"evaluating {estimator} on {dataset}")

    predictor = estimator.train(dataset.train)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )

    pprint.pprint(agg_metrics)

    eval_dict = agg_metrics
    eval_dict["dataset"] = dataset_name
    eval_dict["estimator"] = type(estimator).__name__
    return eval_dict

In [31]:
datasets

['m4_quarterly', 'm4_yearly']

In [32]:
seed = 42
if __name__ == "__main__":
    
    mx.random.seed(seed)
    np.random.seed(seed)
    
    results = []
    for dataset_name in datasets:
        for estimator in estimators:
            print(estimator)
            # catch exceptions that are happening during training to avoid failing the whole evaluation
            try:
                results.append(evaluate(dataset_name, estimator))
            except Exception as e:
                print(str(e))

    df = pd.DataFrame(results)

    sub_df = df[
        [
            "dataset",
            "estimator",
            "RMSE",
            "mean_wQuantileLoss",
            "MASE",
            "sMAPE",
            "MSIS",
        ]
    ]

    print(sub_df.to_string())

functools.partial(<class 'gluonts.model.deepar._estimator.DeepAREstimator'>, trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=None, epochs=200, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=50, patience=10, weight_decay=1e-08))


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_quarterly.


evaluating gluonts.model.deepar._estimator.DeepAREstimator(cardinality=[24000], cell_type="lstm", context_length=None, distr_output=gluonts.distribution.student_t.StudentTOutput(), dropout_rate=0.1, embedding_dimension=None, freq="3M", lags_seq=None, num_cells=40, num_layers=2, num_parallel_samples=100, prediction_length=8, scaling=True, time_features=None, trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=None, epochs=200, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=50, patience=10, weight_decay=1e-08), use_feat_dynamic_real=False, use_feat_static_cat=True, use_feat_static_real=False) on TrainDatasets(metadata=<MetaData freq='3M' target=None feat_static_cat=[<CategoricalFeatureInfo name='feat_static_cat' cardinality='24000'>] feat_static_real=[] feat_dynamic_real=[] feat_dynamic_cat=[] prediction_length=8>, train=<gluonts.dataset.common.FileDataset object at 0x0000023E

INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:06<00:00,  8.25it/s, avg_epoch_loss=8.26]
INFO:root:Epoch[0] Elapsed time 6.103 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=8.259157
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 10.06it/s, avg_epoch_loss=7.63]
INFO:root:Epoch[1] Elapsed time 4.977 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.625053
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 10.85it/s, avg_epoch_loss=7.58]
INFO:root:Epoch[2] Elapsed time 4.617 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.577501
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 10.28it/s, avg_epoch_loss=7.27]
INFO:root:Epoch[3] Elapsed time 4.875 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=7.268690
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.08it/s, avg_epoch_loss=

INFO:root:Epoch[36] Elapsed time 4.742 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=6.838238
INFO:root:Epoch[37] Learning rate is 0.0005
100%|██████████| 50/50 [00:06<00:00,  7.17it/s, avg_epoch_loss=6.96]
INFO:root:Epoch[37] Elapsed time 6.988 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=6.964667
INFO:root:Epoch[38] Learning rate is 0.0005
100%|██████████| 50/50 [00:08<00:00,  6.00it/s, avg_epoch_loss=6.73]
INFO:root:Epoch[38] Elapsed time 8.342 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=6.730379
INFO:root:Epoch[39] Learning rate is 0.0005
100%|██████████| 50/50 [00:05<00:00,  9.18it/s, avg_epoch_loss=6.89]
INFO:root:Epoch[39] Elapsed time 5.460 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=6.891204
INFO:root:Epoch[40] Learning rate is 0.0005
100%|██████████| 50/50 [00:05<00:00,  9.21it/s, avg_epoch_loss=6.93]
INFO:root:Epoch[40] Elapsed time 5.438 seconds
INFO:root:Epoch[40] Evaluation metric 'epoch_loss'=6.928730
INFO:root:Epo

...

INFO:root:Epoch[72] Elapsed time 4.974 seconds
INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=7.001679
INFO:root:Epoch[73] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 12.16it/s, avg_epoch_loss=7.03]
INFO:root:Epoch[73] Elapsed time 4.124 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=7.030619
INFO:root:Epoch[74] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 11.54it/s, avg_epoch_loss=6.46]
INFO:root:Epoch[74] Elapsed time 4.336 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=6.459123
INFO:root:Epoch[75] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 11.92it/s, avg_epoch_loss=6.04]
INFO:root:Epoch[75] Elapsed time 4.205 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=6.042908
INFO:root:Epoch[76] Learning rate is 5e-05
100%|██████████| 50/50 [00:05<00:00,  9.75it/s, avg_epoch_loss=6.24]
INFO:root:Epoch[76] Elapsed time 5.137 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=6.237495
INFO:root:Epoch[7

INFO:root:Epoch[109] Evaluation metric 'epoch_loss'=7.575094
INFO:root:Epoch[110] Learning rate is 5e-05
100%|██████████| 50/50 [00:05<00:00,  9.51it/s, avg_epoch_loss=7.12]
INFO:root:Epoch[110] Elapsed time 5.268 seconds
INFO:root:Epoch[110] Evaluation metric 'epoch_loss'=7.116364
INFO:root:Epoch[111] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 11.72it/s, avg_epoch_loss=6.93]
INFO:root:Epoch[111] Elapsed time 4.278 seconds
INFO:root:Epoch[111] Evaluation metric 'epoch_loss'=6.928608
INFO:root:Epoch[112] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 11.68it/s, avg_epoch_loss=6.75]
INFO:root:Epoch[112] Elapsed time 4.292 seconds
INFO:root:Epoch[112] Evaluation metric 'epoch_loss'=6.750981
INFO:root:Epoch[113] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 12.04it/s, avg_epoch_loss=7.12]
INFO:root:Epoch[113] Elapsed time 4.161 seconds
INFO:root:Epoch[113] Evaluation metric 'epoch_loss'=7.122627
INFO:root:Epoch[114] Learning rate is 5e-05
100%|██

INFO:root:Epoch[146] Elapsed time 5.378 seconds
INFO:root:Epoch[146] Evaluation metric 'epoch_loss'=6.543558
INFO:root:Epoch[147] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 12.47it/s, avg_epoch_loss=5.95]
INFO:root:Epoch[147] Elapsed time 4.017 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=5.948499
INFO:root:Epoch[148] Learning rate is 5e-05
100%|██████████| 50/50 [00:05<00:00,  9.92it/s, avg_epoch_loss=5.99]
INFO:root:Epoch[148] Elapsed time 5.053 seconds
INFO:root:Epoch[148] Evaluation metric 'epoch_loss'=5.990615
INFO:root:Epoch[149] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 10.46it/s, avg_epoch_loss=6.7]
INFO:root:Epoch[149] Elapsed time 4.796 seconds
INFO:root:Epoch[149] Evaluation metric 'epoch_loss'=6.700822
INFO:root:Epoch[150] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 11.62it/s, avg_epoch_loss=6.94]
INFO:root:Epoch[150] Elapsed time 4.315 seconds
INFO:root:Epoch[150] Evaluation metric 'epoch_loss'=6.940289
INFO

100%|██████████| 50/50 [00:04<00:00, 11.80it/s, avg_epoch_loss=6.81]
INFO:root:Epoch[183] Elapsed time 4.247 seconds
INFO:root:Epoch[183] Evaluation metric 'epoch_loss'=6.813986
INFO:root:Epoch[184] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 11.69it/s, avg_epoch_loss=6.57]
INFO:root:Epoch[184] Elapsed time 4.290 seconds
INFO:root:Epoch[184] Evaluation metric 'epoch_loss'=6.573523
INFO:root:Epoch[185] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 11.24it/s, avg_epoch_loss=6.91]
INFO:root:Epoch[185] Elapsed time 4.457 seconds
INFO:root:Epoch[185] Evaluation metric 'epoch_loss'=6.912158
INFO:root:Epoch[186] Learning rate is 5e-05
100%|██████████| 50/50 [00:05<00:00,  9.20it/s, avg_epoch_loss=6.92]
INFO:root:Epoch[186] Elapsed time 5.449 seconds
INFO:root:Epoch[186] Evaluation metric 'epoch_loss'=6.922295
INFO:root:Epoch[187] Learning rate is 5e-05
100%|██████████| 50/50 [00:05<00:00,  8.35it/s, avg_epoch_loss=6.65]
INFO:root:Epoch[187] Elapsed time 5.999 sec

{'Coverage[0.1]': 0.28051449275362533,
 'Coverage[0.2]': 0.3924782608695643,
 'Coverage[0.3]': 0.4667318840579698,
 'Coverage[0.4]': 0.5267536231884081,
 'Coverage[0.5]': 0.57753623188406,
 'Coverage[0.6]': 0.6193913043478287,
 'Coverage[0.7]': 0.6652971014492813,
 'Coverage[0.8]': 0.7147246376811651,
 'Coverage[0.9]': 0.778297101449279,
 'MAE_Coverage': 0.11167632850241455,
 'MASE': 3.4230265632576518,
 'MSE': 3354678.8945107274,
 'MSIS': 65.37920658859298,
 'ND': 0.13912222536963118,
 'NRMSE': 0.29365696696837057,
 'QuantileLoss[0.1]': 66670408.76365204,
 'QuantileLoss[0.2]': 89740383.80376588,
 'QuantileLoss[0.3]': 104569307.64584044,
 'QuantileLoss[0.4]': 114208667.31791994,
 'QuantileLoss[0.5]': 119745932.85394287,
 'QuantileLoss[0.6]': 121482755.63625489,
 'QuantileLoss[0.7]': 119030138.42207947,
 'QuantileLoss[0.8]': 111062143.81912841,
 'QuantileLoss[0.9]': 93460516.064917,
 'RMSE': 1831.5782523579842,
 'abs_error': 119745932.8656311,
 'abs_target_mean': 6237.135359895143,
 'ab

In [33]:
df[["dataset", "estimator", "MASE", "sMAPE", "MSIS","wQuantileLoss[0.5]", "wQuantileLoss[0.9]", ]]

,dataset,estimator,MASE,sMAPE,MSIS,wQuantileLoss[0.5],wQuantileLoss[0.9]
0,m4_quarterly,DeepAREstimator,1.298074,0.107655,14.816202,0.100453,0.059837
1,m4_yearly,DeepAREstimator,3.423027,0.143085,65.379207,0.139122,0.108584
